In [3]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering


In [4]:

# ✅ โหลดโมเดล QA
#model_name = "deepset/xlm-roberta-base-squad2"
#qa_pipeline = pipeline("question-answering", model=model_name, tokenizer=model_name)

# ✅ โหลด test data
test_df = pd.read_csv("test.csv")  # ต้องมี column: id, file, question
submission = []


In [5]:

# ✅ Loop หา context + ตอบคำถาม
for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    file = row["file"]
    question = row["question"]
    context_path = os.path.join("context", file)
    
    if not os.path.exists(context_path):
        submission.append({"id": row["id"], "answer": ""})
        continue

    with open(context_path, encoding="utf-8") as f:
        context = f.read()

    try:
        result = qa_pipeline({
            "context": context,
            "question": question
        })

        # ถ้า score ต่ำหรือไม่มีคำตอบ → ตอบว่าง
        if result["score"] < 0.2 or result["answer"].strip() == "":
            answer = ""
        else:
            answer = result["answer"].strip()

    except Exception as e:
        answer = ""

    submission.append({"id": row["id"], "answer": answer})

# ✅ Save submission
submission_df = pd.DataFrame(submission)
submission_df.to_csv("submission.csv", index=False)
print("✅ Done! Saved to submission.csv")


  0%|          | 0/275 [00:00<?, ?it/s]


KeyError: 'file'